# 1. 라이브러리 불러오기

In [1]:
import platform

import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.4f}'.format

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:
    plt.rcParams['font.family'] = 'AppleGothic'

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [2]:
# 클러스터링 결과 데이터 불러오기
cluster = pd.read_csv('./data/Cluster Result.csv', encoding = 'UTF-8')

# 사용할 변수만 저장
cluster = cluster[['자치구', '행정동', '전체점포수', '주거건물대지면적', '인구지수', 'H_cluster', 'Km_cluster', 'Kmed_cluster']]

# '대치4동', '상도1동' 제거
cluster = cluster[~cluster['행정동'].isin(['대치4동', '상도1동'])]

# 클러스터링 결과값 정수형으로 저장
cluster['H_cluster'] = cluster['H_cluster'].astype('int')
cluster['Km_cluster'] = cluster['Km_cluster'].astype('int')
cluster['Kmed_cluster'] = cluster['Kmed_cluster'].astype('int')

# 데이터 일부 확인
print('Shape of Clustering Result data :', cluster.shape, '\n')
display(cluster.head())

Shape of Clustering Result data : (422, 8) 



,자치구,행정동,전체점포수,주거건물대지면적,인구지수,H_cluster,Km_cluster,Kmed_cluster
0,강남구,개포1동,4,264667.2500,-3.7440,1,4,1
1,강남구,개포2동,19,433676.9400,-1.5539,1,4,1
2,강남구,개포4동,35,378362.6130,-1.3830,1,4,1
3,강남구,논현1동,143,730041.6780,1.7706,2,5,2
4,강남구,논현2동,194,935957.9925,0.2371,3,3,2


In [3]:
model_data = cluster.copy()

standard = StandardScaler()

model_data[['전체점포수', '주거건물대지면적']] = standard.fit_transform(model_data[['전체점포수', '주거건물대지면적']])

model_data.head()

,자치구,행정동,전체점포수,주거건물대지면적,인구지수,H_cluster,Km_cluster,Kmed_cluster
0,강남구,개포1동,-0.7477,-0.7177,-3.7440,1,4,1
1,강남구,개포2동,-0.4880,-0.3909,-1.5539,1,4,1
2,강남구,개포4동,-0.2111,-0.4979,-1.3830,1,4,1
3,강남구,논현1동,1.6583,0.1822,1.7706,2,5,2
4,강남구,논현2동,2.5411,0.5804,0.2371,3,3,2


In [4]:
# Hierarchical Clustering 결과
H_cluster = pd.pivot_table(cluster, index = 'H_cluster', values = ['전체점포수', '주거건물대지면적', '인구지수'], aggfunc = 'mean')

# 3개의 변수의 순위를 구한 뒤, 순위의 합을 구하고 다시 순위를 선정해준다.
H_cluster['순위'] = H_cluster.rank(ascending = False).sum(axis = 1).rank().astype('int')

# 순위 변수를 사용하여 데이터 정렬
H_cluster = H_cluster

# 6번군집이 높은 값을 가지고 있음
h_list = cluster[cluster['H_cluster'] == 6]['행정동'].unique().tolist()

H_cluster.style.background_gradient(cmap = 'Blues', axis = 0)

,인구지수,전체점포수,주거건물대지면적,순위
H_cluster,,,,
1,-1.497298,25.860335,424577.046221,10
2,0.649318,118.739130,1263918.247513,6
3,1.191844,203.714286,751055.452186,4
4,1.350419,39.818182,689825.585504,7
5,-3.338908,109.750000,499165.910450,9
6,1.820133,584.500000,1465698.048000,1
7,0.654459,60.000000,3889904.348000,4
8,1.380290,260.000000,4435491.396500,2
9,-1.341289,339.000000,1025711.810000,3


In [5]:
# K-Means Clustering 결과
Km_cluster = pd.pivot_table(cluster, index = 'Km_cluster', values = ['전체점포수', '주거건물대지면적', '인구지수'], aggfunc = 'mean')

# 3개의 변수의 순위를 구한 뒤, 순위의 합을 구하고 다시 순위를 선정해준다.
Km_cluster['순위'] = Km_cluster.rank(ascending = False).sum(axis = 1).rank().astype('int')

# 2번 군집이 높은 값을 가지고 있음
km_list = cluster[cluster['Km_cluster'] == 2]['행정동'].unique().tolist()

Km_cluster.style.background_gradient(cmap = 'Blues', axis = 0)

,인구지수,전체점포수,주거건물대지면적,순위
Km_cluster,,,,
1,2.366612,45.777778,737081.694756,3
2,1.820133,584.500000,1465698.048000,1
3,0.273448,222.923077,1423195.125662,2
4,-1.879252,26.588652,405210.598855,6
5,0.616075,106.235294,1172733.564168,3
6,0.153961,30.760563,588285.407523,5


In [6]:
# K-Medoids Clustering 결과
Kmed_cluster = pd.pivot_table(cluster, index = 'Kmed_cluster', values = ['전체점포수', '주거건물대지면적', '인구지수'], aggfunc = 'mean')

# 3개의 변수의 순위를 구한 뒤, 순위의 합을 구하고 다시 순위를 선정해준다.
Kmed_cluster['순위'] = Kmed_cluster.rank(ascending = False).sum(axis = 1).rank().astype('int')

# 6번 군집이 높은 값을 가지고 있음
kmed_list = cluster[cluster['Kmed_cluster'] == 6]['행정동'].unique().tolist()

Kmed_cluster.style.background_gradient(cmap = 'Blues', axis = 0)

,인구지수,전체점포수,주거건물대지면적,순위
Kmed_cluster,,,,
1,-2.212987,27.524752,361834.893294,6
2,0.509232,141.888889,1202358.061362,3
3,-0.535644,28.406504,520250.132801,5
4,1.075440,33.032258,656298.668398,4
5,2.773257,52.000000,856984.020181,2
6,1.820133,584.500000,1465698.048000,1


In [7]:
# 3가지 기법의 군집분석 수행 후, 각 기법에서 선택한 군집에 속하는 행정동 확인
print('Hierarchical Clustering :', h_list, '\n')
print('K-means Clustering :', km_list, '\n')
print('K-medoids Clustering :', kmed_list)

Hierarchical Clustering : ['역삼1동', '서교동'] 

K-means Clustering : ['역삼1동', '서교동'] 

K-medoids Clustering : ['역삼1동', '서교동']
